In [0]:
select
  u.notebookid,
  u.service_name,
  u.clusterid,
  u.email,
--  concat(s.table_schema,'.',s.table_name) as datatable,
  max(u.event_time) as most_recent_use,
  count(distinct s.column_name) as column_count,
  column_count - count(distinct s.column_name) as count_unused,
  if(column_count=count_unused,True,False) as table_unused,
  round(count_unused/column_count*100,1) as percent_unused,
  array_intersect(u.columns, collect_set(column_name) ) as unused_columns,
  collect_set(s.column_full_name) as all_columns
from
 edav_prd_cdh.cdh_engineering_etl.product_analytics_cube_collected_columns u
 left join edav_prd_cdh.cdh_engineering_etl.product_analytics_schemas_lineage s
 on s.notebookid=u.notebookid --and
-- s.table_name=u.table_name
--where table_schema in
--  ('cdh_truveta','cdh_truveta_lava',
--  'cdh_premier_lava_dev', 'cdh_premier_v2',
--  'cdh_premier_lava','cdh_premier_lava_dev')
group by
  u.notebookid,
  u.service_name,
  u.clusterid,
  u.email,
  u.columns,
  column_count
--  s.table_schema,
--  s.table_name
order by
  count_unused desc --  ) x